In [2]:
# Dashboard
import plotly
import plotly.express as px
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Classicals
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_validate

In [3]:
# Preperation
df = pd.read_csv('dashboard_airlinequality.csv')
cols_15 = ['Seat Comfort', 'Cabin Staff Service', 'Ground Service', 
            'Value For Money', 'Food & Beverages', 'Inflight Entertainment']
li_seat_type = ['Economy Class', 'Business Class', 'Premium Economy', 'First Class']
li_type_of_traveller = ['Solo Leisure', 'Business', 'Family Leisure', 'Couple Leisure']
li_flight_length = ['Short-Haul', 'Middle-Haul', 'Long-Haul']

In [6]:
# A function wrapping up the chart
def output_chart(df = df,
                 seat_type = ['Economy Class'], 
                 type_of_travaller = ['Solo Leisure'], 
                 stop = [False],
                 flight_length = ['Short-Haul']):
    
    # Select data by category
    mask = (df['Seat Type'].isin(seat_type)) &\
            (df['Type Of Traveller'].isin(type_of_travaller)) &\
            (df['Is_stop'].isin(stop)) &\
            (df['Flight Length'].isin(flight_length))
    df = df[mask]
    
    # Fill nans with mean
    df = df.fillna(df.mean())
            
    # Make X and y
    X = df[cols_15]
    y = df['Recommended']
    
    # Calculate coefficients
    model = linear_model.LogisticRegression(solver='lbfgs', C=0.05)
    model.fit(X, y)
    df_coef = pd.DataFrame({'Feature':X.columns.to_list(),
                        'Coef':model.coef_.tolist()[0]})
    df_coef = df_coef.sort_values(['Coef'])
    df_coef['Importance'] = np.exp(df_coef['Coef']) - 1
    
    # Calculate some statistics
    model_accuracy = cross_validate(model, X, y, cv=5, return_train_score=True)['test_score'].mean()
    n_samples = X.shape[0]
    print(model_accuracy, n_samples)
    
    # Visualize coefficients
    # Intepretation: If the coefficient for Inflight Entertainment is 0.2,
    #                an increase in a star in Inflight Entertainment will make a customer 22% (exp(0.2)=1.22) 
    #                more likely to recommend the flight 
    fig = px.bar(df_coef, y='Feature', x='Importance', color='Feature', orientation='h')
    
    fig.update_layout(title='What Contribute to Customer Satisfaction',
                      yaxis_title='Aspects',
                      xaxis_title='An increase in 1 star makes a customer X% more likely to recommend a flight',
                      xaxis_tickformat = ',.0%',
                      showlegend=False,
                      font={'size':16})
    
    return fig
    

In [21]:
# Running the dashboard

external_stylesheets = [dbc.themes.BOOTSTRAP]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    dbc.Row([
        dbc.Col([
                html.H6('Choose your viz'),
            
                html.Label('Class'),
                dcc.Dropdown(
                id='class', 
                options=[{'label': i, 'value': i} for i in li_seat_type],
                multi=True,
                value=['Economy Class']), 

                html.Label('Travel Purpose'),
                dcc.Dropdown(
                id='purpose',    
                options=[{'label': i, 'value': i} for i in li_type_of_traveller],
                multi=True,
                value=['Solo Leisure']),

                html.Label('Flight Length'),
                dcc.Dropdown(
                id='length',    
                options=[{'label': i.replace('-',' '), 'value': i} for i in li_flight_length],
                multi=True,
                value=['Short-Haul']),

                html.Label('Is there a stop'),
                dcc.Dropdown(
                id='stop',    
                options=[{'label': 'Non-stop', 'value': True},
                         {'label': 'Stop', 'value': False}],
                multi=True,
                value=[True])],md=4),
        
        dbc.Col([dcc.Graph(id='chart')])
    
    ]))

@app.callback(
    Output('chart', 'figure'),
    [Input('class', 'value'),
     Input('purpose', 'value'),
     Input('length', 'value'),
     Input('stop', 'value')])
def update_figure(seat_type, type_of_travaller, flight_length, stop):
    return output_chart(df, seat_type, type_of_travaller, stop, flight_length)

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jan/2020 22:07:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jan/2020 22:07:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jan/2020 22:07:02] "GET /_dash-layout HTTP/1.1" 200 -


0.9255555555555557 96


127.0.0.1 - - [28/Jan/2020 22:07:02] "POST /_dash-update-component HTTP/1.1" 200 -


0.9309022556390978 102


...

127.0.0.1 - - [28/Jan/2020 22:07:10] "POST /_dash-update-component HTTP/1.1" 200 -


0.9495858532686942 3015


127.0.0.1 - - [28/Jan/2020 22:07:12] "POST /_dash-update-component HTTP/1.1" 200 -


0.9440613411551235 10654


127.0.0.1 - - [28/Jan/2020 22:07:14] "POST /_dash-update-component HTTP/1.1" 200 -
